In [51]:
import pandas as pd
import requests
from collections import defaultdict
from pathlib import Path

In [52]:
year = 2021
leaderboard_id = 976545
token_path = Path.home() / '.config/aocd/token'

In [53]:
with open(token_path) as f:
    token = f.read().strip()

cookies = {'session': token}
response = requests.get(f'https://adventofcode.com/{year}/leaderboard/private/view/{leaderboard_id}.json', cookies=cookies)
j = response.json()

In [54]:
time_diffs = defaultdict(lambda: defaultdict(str))

for id, member in j['members'].items():
    name = member['name'] if member['name'] else id
    for day, times in member['completion_day_level'].items():
        if '1' in times and '2' in times:
            time_diffs[name][int(day)] = int(times['2']['get_star_ts']) - int(times['1']['get_star_ts'])

In [55]:
df = pd.DataFrame.from_dict(time_diffs, orient='index')
df.sort_index(key=lambda x: x.str.lower(), inplace=True)
df.sort_index(axis=1, inplace=True)

In [56]:
df

,1,2,3
1014573,963.0,246.0,1933.0
1047131,NaN,245.0,NaN
1139456,808.0,151.0,16775.0
1495816,4343.0,219.0,NaN
1525642,1379.0,240.0,2014.0
1547063,51612.0,316.0,NaN
1603729,135.0,655.0,9390.0
1614144,16.0,1407.0,65.0
1625081,1848.0,553.0,NaN
3lacki,2123.0,18.0,NaN


In [57]:
num_members, num_days = df.shape
points = pd.Series(index=df.index, dtype=int)

for day in range(1, num_days+1):
    max_points = num_members
    for name, time in df[day].sort_values().iteritems():
        if not pd.isna(time):
            points[name] += max_points
            max_points -= 1

In [58]:
points.sort_values(ascending=False)

Sanjay                  139
devgertschi             129
attoPascal              126
underway                124
stuck_in_head           124
S-ecki                  112
1614144                 105
felpower                103
Triarier                102
zoidberg77               96
NotsoblackReaper         95
thoosequa                95
michalrzak               95
KaNaDaAT                 95
MorThu                   95
David Wild               88
Hainkinho                87
Ebit Daddy               85
Katakompe                85
1014573                  84
RaphaelTarita            84
1525642                  82
Salominius               81
1139456                  79
marc1729                 78
999754                   76
Grarak                   73
Der_Himbeerkuchen        73
1603729                  72
Dominik                  72
Mike15423                69
Hablam                   69
st1fado                  68
Samuel Šulovský          58
3lacki                   57
hilmar05            